In [16]:
from pyspark.sql import SparkSession
sc=SparkSession.builder.getOrCreate().sparkContext

In [20]:
input=sc.textFile('logs.txt').persist()

In [21]:
# метод toDebugString () применяется для вывода иерархии происхождения наборов RDD в Spark, т.е. что за чем выполнялось
input.toDebugString()

b'(2) logs.txt MapPartitionsRDD[51] at textFile at <unknown>:0 []\n |  logs.txt HadoopRDD[50] at textFile at <unknown>:0 []'

In [29]:
def func(words):
    if len(words)>0:
        return words
    return ''
tokenized=input.map(lambda line: line.split(' ')).filter(func).persist()

In [30]:
tokenized.toDebugString()

b'(2) PythonRDD[61] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]\n |  logs.txt MapPartitionsRDD[51] at textFile at <unknown>:0 [Memory Serialized 1x Replicated]\n |  logs.txt HadoopRDD[50] at textFile at <unknown>:0 [Memory Serialized 1x Replicated]'

In [31]:
tokenized.collect()

[['#', 'input.', 'txt'],
 ['INFO', 'This', 'is', 'а', 'message', 'with', 'content'],
 ['INFO', 'This', 'is', 'some', 'other', 'content'],
 [''],
 ['INFO', 'Неге', 'are', 'more', 'messages'],
 ['WARN', 'This', 'is', 'а', 'warning'],
 [''],
 ['ERROR', 'Something', 'bad', 'happened'],
 ['WARN', 'More', 'details', 'on', 'the', 'bad', 'thing'],
 ['INFO', 'back', 'to', 'normal', 'messages']]

In [8]:
# Данная последовательность команд создает набор RDD counts с числом записей для каждой степени важности
count=tokenized.map(lambda words: (words[0], 1)).reduceByKey(lambda a,b: a+b).persist()

In [15]:
count.toDebugString()

b'(2) PythonRDD[46] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]\n |       CachedPartitions: 2; MemorySize: 196.0 B; DiskSize: 0.0 B\n |  MapPartitionsRDD[45] at mapPartitions at PythonRDD.scala:145 [Memory Serialized 1x Replicated]\n |  ShuffledRDD[44] at partitionBy at <unknown>:0 [Memory Serialized 1x Replicated]\n +-(2) PairwiseRDD[43] at reduceByKey at C:\\Users\\golik\\AppData\\Local\\Temp\\ipykernel_14648\\4181502713.py:1 [Memory Serialized 1x Replicated]\n    |  PythonRDD[42] at reduceByKey at C:\\Users\\golik\\AppData\\Local\\Temp\\ipykernel_14648\\4181502713.py:1 [Memory Serialized 1x Replicated]\n    |  PythonRDD[9] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]\n    |      CachedPartitions: 2; MemorySize: 460.0 B; DiskSize: 0.0 B\n    |  logs.txt MapPartitionsRDD[8] at textFile at <unknown>:0 [Memory Serialized 1x Replicated]\n    |      CachedPartitions: 2; MemorySize: 306.0 B; DiskSize: 0.0 B\n    |  logs.txt HadoopRDD[7] at textFile at <

In [10]:
count.collect()

[('#', 1), ('INFO', 4), ('', 2), ('WARN', 2), ('ERROR', 1)]